# Demo for Unconditional Generation

In [1]:
# Designate GPU to use
import os
gpu_ids = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_ids}"

In [2]:
# import libraries
import numpy as np
from PIL import Image
from pathlib import Path
from IPython.display import Image as ipy_image
from IPython.display import display
from termcolor import colored, cprint

import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import torchvision.utils as vutils
import torch.nn.functional as F

from datasets.dataloader import CreateDataLoader, get_data_generator

from collections import OrderedDict
from models.base_model import create_model

import joblib

from visualize.visualizer import MOFVisualizer, animate, resize_gif, make_concat_gif
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import glob
import os

from utils.data_util import lattices_to_params_shape, get_gt_crys_ori

import warnings
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from torch_geometric.data import Batch, Data

%load_ext autoreload
%autoreload 2

/home/parkjunkil/anaconda3/envs/chgdiff_2/lib/python3.10/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


In [3]:
# Options for the model. please check `utils/demo_util.py` for more details
from utils.demo_util import ChargeDIFFOpt

seed = 42
opt = ChargeDIFFOpt(gpu_ids=gpu_ids, seed=seed)

opt.init_dset_args()
opt.init_model_args()

opt.batch_size = 32

device = opt.device

[*] CHGDIFF_TestOption initialized.


In [4]:
# Initialize ChargeDIFF model

opt.model='chargediff_bandgap'
opt.vq_cfg="./configs/vqvae.yaml"
opt.vq_ckpt="./saved_ckpt/vqvae.pth"
opt.df_cfg = './configs/chargediff_cond.yaml'
opt.ckpt = './saved_ckpt/bandgap.pth'

CHARGEDIFF = create_model(opt)
cprint(f'[*] "{CHARGEDIFF.name()}" loaded.', 'cyan')

[*] VQVAE: weight successfully load from: ./saved_ckpt/vqvae.pth
[*] weight successfully load from: ./saved_ckpt/bandgap.pth
[*] Model has been created: CHARGEDIFF-Model
[*] "CHARGEDIFF-Model" loaded.


In [5]:
# Only for cases of conditioning scalar properties
from datasets.base_dataset import CreateDataset

opt.dataset_mode = 'MP-20-Charge'

train_dataset, val_dataset, test_dataset = CreateDataset(opt)
scaler = train_dataset.scaler

[*] Data Processing Start
[*] Could a take while if this is the first run
[*] 36465 samples loaded for train.
[*] 4051 samples loaded for val.
[*] 4051 samples loaded for test.
[*] Data Processing End
[*] Dataset has been created: CHARGEDIFF_Dataset-MP-20-Charge


In [6]:
from pymatgen.io.vasp.outputs import Chgcar, VolumetricData

batches = 2
batch_size = 50

for i in range(batches):
    
    gen_crys, chgden_tot = CHARGEDIFF.cond(batch_size = batch_size, scaler=scaler, target=3.0, dataset='mp_20_charge', cutoff = 0.5)


    for j in range(len(gen_crys)):

        cry = gen_crys[j]
        chgden = chgden_tot[j]
        
        structure = cry.structure
        
        volumetric = VolumetricData(structure = structure, data={'total': chgden})

        
        
        idx = i*batch_size + j

        structure.to_file(f'./sample/bandgap/3.0/cifs/{900+idx}.cif')
        volumetric.write_file(f'./sample/bandgap/3.0/vasp/{900+idx}.vasp')



100%|██████████| 1000/1000 [02:44<00:00,  6.06it/s]


In [7]:
from pymatgen.io.vasp.outputs import Chgcar, VolumetricData

batches = 20
batch_size = 50

for i in range(batches):
    
    gen_crys, chgden_tot = CHARGEDIFF.cond(batch_size = batch_size, scaler=scaler, target=4.0, dataset='mp_20_charge', cutoff = 0.5)


    for j in range(len(gen_crys)):

        cry = gen_crys[j]
        chgden = chgden_tot[j]
        
        structure = cry.structure
        
        volumetric = VolumetricData(structure = structure, data={'total': chgden})

        
        
        idx = i*batch_size + j

        structure.to_file(f'./sample/bandgap/4.0/cifs/{idx}.cif')
        volumetric.write_file(f'./sample/bandgap/4.0/vasp/{idx}.vasp')



100%|██████████| 1000/1000 [02:45<00:00,  6.04it/s]
